<a href="https://colab.research.google.com/github/purnawanpp/gnina10/blob/main/gnina_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jika sudah dibuat pertama kali untuk berikut perintah ini tidak perlu dijalankan

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Download file dari GitHub
!wget https://github.com/gnina/gnina/releases/download/v1.3.2/gnina.1.3.2 -O gnina1.3.2

# Pindahkan file ke Google Drive (misal ke folder MyDrive/gnina/)
!mv gnina1.3.2 "/content/drive/MyDrive/gnina1.3.2"


Mounted at /content/drive
--2025-11-27 22:11:16--  https://github.com/gnina/gnina/releases/download/v1.3.2/gnina.1.3.2
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/45548146/415ca903-8da5-4152-ba94-5a0d1d3dbcf5?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-27T23%3A03%3A22Z&rscd=attachment%3B+filename%3Dgnina.1.3.2&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-27T22%3A02%3A57Z&ske=2025-11-27T23%3A03%3A22Z&sks=b&skv=2018-11-09&sig=hJYbC7TMUwn84PuZZa3mc%2Fav88zpXxV3%2FTD5luMnfls%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDI4NTA3NywibmJmIjoxNzY0MjgxNDc3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZ

In [12]:
# Mount drive (jika belum)
from google.colab import drive
drive.mount('/content/drive')

# Path ke GNINA Anda
gnina_path = "/content/drive/MyDrive/gnina1.3.2"

# Jadikan executable
!chmod +x "$gnina_path"

# Buat symlink ke /usr/local/bin
!ln -sf "$gnina_path" /usr/local/bin/gnina


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!nvidia-smi

Thu Nov 27 22:27:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!apt install openbabel
!pip install py3Dmol

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libboost-iostreams1.74.0 libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libboost-iostreams1.74.0 libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 4,148 kB of archives.
After this operation, 19.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libboost-iostreams1.74.0 amd64 1.74.0-14ubuntu3 [245 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel am

In [31]:
!LC_ALL=C gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb -o docked_native.mol2 --seed 0 --cnn_verbose > redocking.txt

*** Open Babel Warning  in Init
  Cannot initialize database 'space-groups.txt' which may cause further errors.


In [27]:
!LC_ALL=C obrms -f lig.pdb docked_native.mol2 > rmsd.txt

In [30]:
import glob
import os
import subprocess

# ==============================
# KONFIGURASI DASAR
# ==============================
receptor = "rec.pdb"          # reseptor
autobox_ligand = "lig.pdb"    # ligand referensi untuk autobox
ligand_pattern = "lig*.mol2"  # pola nama: lig1.mol2, lig2.mol2, ...

seed = "0"
num_modes = "1"

# ==============================
# CEK FILE WAJIB
# ==============================
if not os.path.exists(receptor):
    raise FileNotFoundError(f"File reseptor tidak ditemukan: {receptor}")

if not os.path.exists(autobox_ligand):
    raise FileNotFoundError(f"File autobox_ligand tidak ditemukan: {autobox_ligand}")

ligand_files = sorted(glob.glob(ligand_pattern))

if not ligand_files:
    raise FileNotFoundError(f"Tidak ada file ligand dengan pola: {ligand_pattern}")

print("Ditemukan ligand:")
for lf in ligand_files:
    print(" -", lf)

# ==============================
# LOOP DOCKING
# ==============================
for lig in ligand_files:
    # contoh: lig = "lig3.mol2"
    base = os.path.splitext(os.path.basename(lig))[0]  # "lig3"
    num = base.replace("lig", "")                     # "3"
    out_mol2 = f"gnina{num}.mol2"                     # "gnina3.mol2"
    out_log  = f"gnina{num}.txt"                      # "gnina3.txt"

    cmd = [
        "gnina",
        "-r", receptor,
        "-l", lig,
        "--autobox_ligand", autobox_ligand,
        "-o", out_mol2,
        "--seed", seed,
        "--num_modes", num_modes,
    ]

    print("\n================================================")
    print(f"Docking {lig}")
    print(f"Output pose : {out_mol2}")
    print(f"Output log  : {out_log}")
    print("Command     :", " ".join(cmd))
    print("================================================")

    # jalankan gnina dan tangkap stdout + stderr
    result = subprocess.run(cmd, text=True, capture_output=True)

    # simpan ke file log
    with open(out_log, "w") as f:
        f.write("### STDOUT ###\n")
        f.write(result.stdout)
        f.write("\n\n### STDERR ###\n")
        f.write(result.stderr)

    # ringkasan di layar
    if result.returncode == 0:
        print(f"✅ Sukses: {lig} → {out_mol2} & {out_log}")
        # opsional: tampilkan baris yang mengandung skor
        for line in result.stdout.splitlines():
            if "minimizedAffinity" in line or "CNN" in line:
                print("   ", line)
    else:
        print(f"❌ Gagal: {lig}")
        print("Periksa log di:", out_log)

print("\nSelesai. Semua ligand telah diproses.")

# ==============================
# PACKING HASIL KE DALAM .ZIP
# ==============================
import zipfile

zip_name = "hasil_docking_gnina.zip"

files_to_zip = []

# semua file gnina*.mol2 dan gnina*.txt
files_to_zip.extend(glob.glob("gnina*.mol2"))
files_to_zip.extend(glob.glob("gnina*.txt"))

# tambahkan rmsd.txt dan docked_native.mol2 jika ada
extra_files = ["rmsd.txt", "docked_native.mol2", "redocking.txt"]
for ef in extra_files:
    if os.path.exists(ef):
        files_to_zip.append(ef)

print("\nFile yang akan dimasukkan ke ZIP:")
for f in files_to_zip:
    print(" -", f)

# buat file zip
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
    for f in files_to_zip:
        zf.write(f)

print(f"\n✅ ZIP selesai dibuat: {zip_name}")

# ==============================
# AUTO-DOWNLOAD .ZIP DI COLAB
# ==============================
from google.colab import files
files.download(zip_name)


Ditemukan ligand:
 - lig1.mol2
 - lig2.mol2

Docking lig1.mol2
Output pose : gnina1.mol2
Output log  : gnina1.txt
Command     : gnina -r rec.pdb -l lig1.mol2 --autobox_ligand lig.pdb -o gnina1.mol2 --seed 0 --num_modes 1
✅ Sukses: lig1.mol2 → gnina1.mol2 & gnina1.txt
    mode |  affinity  |  intramol  |    CNN     |   CNN

Docking lig2.mol2
Output pose : gnina2.mol2
Output log  : gnina2.txt
Command     : gnina -r rec.pdb -l lig2.mol2 --autobox_ligand lig.pdb -o gnina2.mol2 --seed 0 --num_modes 1
✅ Sukses: lig2.mol2 → gnina2.mol2 & gnina2.txt
    mode |  affinity  |  intramol  |    CNN     |   CNN

Selesai. Semua ligand telah diproses.

File yang akan dimasukkan ke ZIP:
 - gnina1.mol2
 - gnina2.mol2
 - gnina2.txt
 - gnina1.txt
 - rmsd.txt
 - docked_native.mol2
 - redocking.txt

✅ ZIP selesai dibuat: hasil_docking_gnina.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>